# Modeling  mid Total Counts

# Purpose

Model by Total Unique counts and produce Coefficients to ../../models/TC_0628_zinb_batchadded.tsv

# Steps


1) Fit ZINB model using Non-Normalized Total Counts 
2) Add Interecept Stderr Cutoff
3) Write Coefficients to ../../models/TC_0628_zinb_batchadded.tsv

# Conclusions

1) Much Improved Results with cutoffs in place

In [1]:
# NOTEBOOK

In [108]:
#Required Packages 

In [1]:
suppressWarnings(suppressMessages(library(tidyverse)))
suppressWarnings(suppressMessages(require(pscl)))
suppressWarnings(suppressMessages(require(MASS)))
suppressWarnings(suppressMessages(require(reshape)))
library(doParallel)
registerDoParallel(3)
suppressWarnings(suppressMessages(require(ordinal)))
suppressWarnings(suppressMessages(require(glmnet)))

Warning message:
"package 'doParallel' was built under R version 4.2.1"
Loading required package: foreach

Warning message:
"package 'foreach' was built under R version 4.2.1"

Attaching package: 'foreach'


The following objects are masked from 'package:purrr':

    accumulate, when


Loading required package: iterators

Warning message:
"package 'iterators' was built under R version 4.2.1"
Loading required package: parallel



In [2]:
# Data Import from Processed file Under data/processed/

In [3]:
tn <- read.table("../../data/processed/TC_mid_production1_2_3_4_5.tsv",header=T,sep="\t")

In [4]:
length(unique(tn$locus_tag))

[1] 4084

In [5]:
tn$batch=as.factor(tn$batch)

In [6]:
colnames(tn)[9] <- "N"

In [7]:
head(tn)

,locus_tag,strain,condition,slevel,rep,batch,tncnt,time,N
,<chr>,<chr>,<chr>,<chr>,<int>,<fct>,<int>,<int>,<int>
1,CCNA_R0094,WT,CONTROL,none,4,4,1,24,1975900
2,CCNA_00001,WT,CONTROL,none,4,4,111,24,1975900
3,CCNA_00002,WT,CONTROL,none,4,4,3786,24,1975900
4,CCNA_00003,WT,CONTROL,none,4,4,534,24,1975900
5,CCNA_00004,WT,CONTROL,none,4,4,0,24,1975900
6,CCNA_00005,WT,CONTROL,none,4,4,0,24,1975900


In [34]:
## TC

In [23]:
nbridge_signif = function(df) {
  # Extract the genename from the model
  locus_tag=as.character(unique(df$locus_tag))

  ## Model starts here
  model = tryCatch(
    {
      x<- model.matrix(tncnt ~ strain+strain:condition+strain:condition:slevel,df)
      y <- df$tncnt
      lambda_seq <- 10^seq(2, -2, by = -.1)
      t<- glm.nb(tncnt ~ strain:condition:slevel,df)$theta
      lambdas = NULL
      for (i in 1:5)
      {
        fit <- cv.glmnet(x, y, alpha = 0, lambda = lambda_seq,family = negative.binomial(theta = t),parallel=TRUE,intercept = FALSE)
        errors = data.frame(fit$lambda,fit$cvm)
        lambdas <- rbind(lambdas,errors)
      }
      # take mean cvm for each lambda
      lambdas <- aggregate(lambdas[, 2], list(lambdas$fit.lambda), mean)

      # select the best one
      bestindex = which(lambdas[2]==min(lambdas[2]))
      best_lambda = lambdas[bestindex,1]
      best_ridge <- glmnet(x, y, alpha = 0, lambda = best_lambda,family = negative.binomial(theta = t),intercept = FALSE)
    },
    error=function(err) {
      status <<- err$message
      return(NULL)
    })


  ## Writing row of NA's if model is null. Usually Coefficient Output has 8 columns. Change this
  if (is.null(coef(model))) {
      model1 = tryCatch(
    {
      x<- model.matrix(tncnt ~ strain+strain:condition+strain:condition:slevel,df)
      y <- df$tncnt
      lambda_seq <- 10^seq(2, -2, by = -.1)
      t<- glm.nb(tncnt ~ strain:condition:slevel,df)$theta
      lambdas = NULL
      for (i in 1:5)
      {
        fit <- cv.glmnet(x, y, alpha = 0, lambda = lambda_seq,family = negative.binomial(theta = t),parallel=TRUE,intercept = FALSE)
        errors = data.frame(fit$lambda,fit$cvm)
        lambdas <- rbind(lambdas,errors)
      }
      # take mean cvm for each lambda
      lambdas <- aggregate(lambdas[, 2], list(lambdas$fit.lambda), mean)

      # select the best one
      bestindex = which(lambdas[2]==min(lambdas[2]))
      best_lambda = lambdas[bestindex,1]
      best_ridge <- glmnet(x, y, alpha = 0, lambda = best_lambda,family = negative.binomial(theta = t),intercept = FALSE)
    },
    error=function(err) {
      status <<- err$message
      return(NULL)
    })
    if (is.null(coef(model1))) {
     namatrix=t(as.matrix(rep("NA",82)))
    row.names(namatrix)= locus_tag
    return (namatrix)}
    else 
        {
    vals<- t(as.matrix(coef(model1)))
    coeffs=t(as.matrix(vals[,-(which(colSums(vals) == 0))]))
    row.names(coeffs)= locus_tag
    return (coeffs) 
    }
    }
  else {
    vals<- t(as.matrix(coef(model)))
    coeffs=t(as.matrix(vals[,-(which(colSums(vals) == 0))]))
    row.names(coeffs)= locus_tag
    return (coeffs)
  }
}

In [24]:
ord_condition<- c("CONTROL","HS","OS","CAN")
ord_strain <- c("WT","DLON","DKJ","DCLPA","DCLPB")
ord_slevel <- c("none","LOW","MEDIUM","HIGH")
df<- tn%>%filter(locus_tag=="CCNA_00001")%>%mutate(strain=fct_relevel(strain,ord_strain))%>%mutate(condition=fct_relevel(condition,ord_condition))%>%mutate(slevel=fct_relevel(slevel,ord_slevel))
a<- suppressWarnings(suppressMessages(nbridge_signif(df)))

In [25]:
model.matrix(tncnt ~ strain+condition+strain:condition:slevel,df)

,(Intercept),strainDLON,strainDKJ,strainDCLPA,strainDCLPB,conditionHS,conditionOS,conditionCAN,strainWT:conditionCONTROL:slevelnone,strainDLON:conditionCONTROL:slevelnone,⋯,strainWT:conditionOS:slevelHIGH,strainDLON:conditionOS:slevelHIGH,strainDKJ:conditionOS:slevelHIGH,strainDCLPA:conditionOS:slevelHIGH,strainDCLPB:conditionOS:slevelHIGH,strainWT:conditionCAN:slevelHIGH,strainDLON:conditionCAN:slevelHIGH,strainDKJ:conditionCAN:slevelHIGH,strainDCLPA:conditionCAN:slevelHIGH,strainDCLPB:conditionCAN:slevelHIGH
1,1,0,0,0,0,0,0,0,1,0,⋯,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,⋯,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,1,0,⋯,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,⋯,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
10,1,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [26]:
colnames(a)

[1] "strainDLON"                           
 [2] "strainDKJ"                            
 [3] "strainDCLPA"                          
 [4] "strainDCLPB"                          
 [5] "strainWT:conditionHS"                 
 [6] "strainDLON:conditionHS"               
 [7] "strainDKJ:conditionHS"                
 [8] "strainDCLPA:conditionHS"              
 [9] "strainDCLPB:conditionHS"              
[10] "strainWT:conditionOS"                 
[11] "strainDLON:conditionOS"               
[12] "strainDKJ:conditionOS"                
[13] "strainDCLPA:conditionOS"              
[14] "strainDCLPB:conditionOS"              
[15] "strainWT:conditionCAN"                
[16] "strainDLON:conditionCAN"              
[17] "strainDKJ:conditionCAN"               
[18] "strainDCLPA:conditionCAN"             
[19] "strainDCLPB:conditionCAN"             
[20] "strainWT:conditionHS:slevelLOW"       
[21] "strainDLON:conditionHS:slevelLOW"     
[22] "strainDKJ:conditionHS:slevelLOW"      
[23] "strainDCLPA:conditionHS:slevelLOW"    
[24] "strainDCLPB:conditionHS:slevelLOW"    
[25] "strainWT:conditionOS:slevelLOW"       
[26] "strainDLON:conditionOS:slevelLOW"     
[27] "strainDKJ:conditionOS:slevelLOW"      
[28] "strainDCLPA:conditionOS:slevelLOW"    
[29] "strainDCLPB:conditionOS:slevelLOW"    
[30] "strainWT:conditionCAN:slevelLOW"      
[31] "strainDLON:conditionCAN:slevelLOW"    
[32] "strainDKJ:conditionCAN:slevelLOW"     
[33] "strainDCLPA:conditionCAN:slevelLOW"   
[34] "strainDCLPB:conditionCAN:slevelLOW"   
[35] "strainWT:conditionHS:slevelMEDIUM"    
[36] "strainDLON:conditionHS:slevelMEDIUM"  
[37] "strainDKJ:conditionHS:slevelMEDIUM"   
[38] "strainDCLPA:conditionHS:slevelMEDIUM" 
[39] "strainDCLPB:conditionHS:slevelMEDIUM" 
[40] "strainWT:conditionOS:slevelMEDIUM"    
[41] "strainDLON:conditionOS:slevelMEDIUM"  
[42] "strainDKJ:conditionOS:slevelMEDIUM"   
[43] "strainDCLPA:conditionOS:slevelMEDIUM" 
[44] "strainDCLPB:conditionOS:slevelMEDIUM" 
[45] "strainWT:conditionCAN:slevelMEDIUM"   
[46] "strainDLON:conditionCAN:slevelMEDIUM" 
[47] "strainDKJ:conditionCAN:slevelMEDIUM"  
[48] "strainDCLPA:conditionCAN:slevelMEDIUM"
[49] "strainDCLPB:conditionCAN:slevelMEDIUM"
[50] "strainWT:conditionHS:slevelHIGH"      
[51] "strainDLON:conditionHS:slevelHIGH"    
[52] "strainDKJ:conditionHS:slevelHIGH"     
[53] "strainDCLPA:conditionHS:slevelHIGH"   
[54] "strainDCLPB:conditionHS:slevelHIGH"   
[55] "strainWT:conditionOS:slevelHIGH"      
[56] "strainDLON:conditionOS:slevelHIGH"    
[57] "strainDKJ:conditionOS:slevelHIGH"     
[58] "strainDCLPA:conditionOS:slevelHIGH"   
[59] "strainDCLPB:conditionOS:slevelHIGH"   
[60] "strainWT:conditionCAN:slevelHIGH"     
[61] "strainDLON:conditionCAN:slevelHIGH"   
[62] "strainDKJ:conditionCAN:slevelHIGH"    
[63] "strainDCLPA:conditionCAN:slevelHIGH"  
[64] "strainDCLPB:conditionCAN:slevelHIGH"

In [34]:
x<- model.matrix(tncnt ~ strain:condition:slevel,df)
      y <- df$tncnt
      lambda_seq <- 10^seq(2, -2, by = -.1)
      t<- glm.nb(tncnt ~ strain:condition:slevel,df)$theta
      lambdas = NULL
      for (i in 1:5)
      {
        fit <- cv.glmnet(x, y, alpha = 0, lambda = lambda_seq,family = negative.binomial(theta = t),parallel=TRUE,intercept = FALSE)
        errors = data.frame(fit$lambda,fit$cvm)
        lambdas <- rbind(lambdas,errors)
      }
      # take mean cvm for each lambda
      lambdas <- aggregate(lambdas[, 2], list(lambdas$fit.lambda), mean)

      # select the best one
      bestindex = which(lambdas[2]==min(lambdas[2]))
      best_lambda = lambdas[bestindex,1]
      best_ridge <- glmnet(x, y, alpha = 0, lambda = best_lambda,family = negative.binomial(theta = t),intercept = FALSE)

ERROR: Error in cv.glmnet(x, y, alpha = 0, lambda = lambda_seq, family = negative.binomial(theta = t), : could not find function "cv.glmnet"


In [20]:
df%>%filter(strain=="DLON")

locus_tag,strain,condition,slevel,rep,batch,tncnt,time,N
<chr>,<fct>,<fct>,<fct>,<int>,<fct>,<int>,<int>,<int>
CCNA_00001,DLON,CONTROL,none,4,4,132,24,3184320
CCNA_00001,DLON,CONTROL,none,3,3,121,24,3610309
CCNA_00001,DLON,CONTROL,none,2,2,438,24,3517574
CCNA_00001,DLON,CONTROL,none,1,1,179,24,2423413
CCNA_00001,DLON,HS,HIGH,4,4,7,24,3195578
CCNA_00001,DLON,HS,HIGH,3,3,11,24,3343918
CCNA_00001,DLON,HS,HIGH,2,2,18,24,2743832
CCNA_00001,DLON,HS,HIGH,1,1,6,24,1390572
CCNA_00001,DLON,HS,MEDIUM,4,4,26,24,3053374


In [29]:
colnames(model.matrix(tncnt ~ strain:condition:slevel,df))

[1] "(Intercept)"                              
 [2] "strainWT:conditionCONTROL:slevelnone"     
 [3] "strainDLON:conditionCONTROL:slevelnone"   
 [4] "strainDKJ:conditionCONTROL:slevelnone"    
 [5] "strainDCLPA:conditionCONTROL:slevelnone"  
 [6] "strainDCLPB:conditionCONTROL:slevelnone"  
 [7] "strainWT:conditionHS:slevelnone"          
 [8] "strainDLON:conditionHS:slevelnone"        
 [9] "strainDKJ:conditionHS:slevelnone"         
[10] "strainDCLPA:conditionHS:slevelnone"       
[11] "strainDCLPB:conditionHS:slevelnone"       
[12] "strainWT:conditionOS:slevelnone"          
[13] "strainDLON:conditionOS:slevelnone"        
[14] "strainDKJ:conditionOS:slevelnone"         
[15] "strainDCLPA:conditionOS:slevelnone"       
[16] "strainDCLPB:conditionOS:slevelnone"       
[17] "strainWT:conditionCAN:slevelnone"         
[18] "strainDLON:conditionCAN:slevelnone"       
[19] "strainDKJ:conditionCAN:slevelnone"        
[20] "strainDCLPA:conditionCAN:slevelnone"      
[21] "strainDCLPB:conditionCAN:slevelnone"      
[22] "strainWT:conditionCONTROL:slevelLOW"      
[23] "strainDLON:conditionCONTROL:slevelLOW"    
[24] "strainDKJ:conditionCONTROL:slevelLOW"     
[25] "strainDCLPA:conditionCONTROL:slevelLOW"   
[26] "strainDCLPB:conditionCONTROL:slevelLOW"   
[27] "strainWT:conditionHS:slevelLOW"           
[28] "strainDLON:conditionHS:slevelLOW"         
[29] "strainDKJ:conditionHS:slevelLOW"          
[30] "strainDCLPA:conditionHS:slevelLOW"        
[31] "strainDCLPB:conditionHS:slevelLOW"        
[32] "strainWT:conditionOS:slevelLOW"           
[33] "strainDLON:conditionOS:slevelLOW"         
[34] "strainDKJ:conditionOS:slevelLOW"          
[35] "strainDCLPA:conditionOS:slevelLOW"        
[36] "strainDCLPB:conditionOS:slevelLOW"        
[37] "strainWT:conditionCAN:slevelLOW"          
[38] "strainDLON:conditionCAN:slevelLOW"        
[39] "strainDKJ:conditionCAN:slevelLOW"         
[40] "strainDCLPA:conditionCAN:slevelLOW"       
[41] "strainDCLPB:conditionCAN:slevelLOW"       
[42] "strainWT:conditionCONTROL:slevelMEDIUM"   
[43] "strainDLON:conditionCONTROL:slevelMEDIUM" 
[44] "strainDKJ:conditionCONTROL:slevelMEDIUM"  
[45] "strainDCLPA:conditionCONTROL:slevelMEDIUM"
[46] "strainDCLPB:conditionCONTROL:slevelMEDIUM"
[47] "strainWT:conditionHS:slevelMEDIUM"        
[48] "strainDLON:conditionHS:slevelMEDIUM"      
[49] "strainDKJ:conditionHS:slevelMEDIUM"       
[50] "strainDCLPA:conditionHS:slevelMEDIUM"     
[51] "strainDCLPB:conditionHS:slevelMEDIUM"     
[52] "strainWT:conditionOS:slevelMEDIUM"        
[53] "strainDLON:conditionOS:slevelMEDIUM"      
[54] "strainDKJ:conditionOS:slevelMEDIUM"       
[55] "strainDCLPA:conditionOS:slevelMEDIUM"     
[56] "strainDCLPB:conditionOS:slevelMEDIUM"     
[57] "strainWT:conditionCAN:slevelMEDIUM"       
[58] "strainDLON:conditionCAN:slevelMEDIUM"     
[59] "strainDKJ:conditionCAN:slevelMEDIUM"      
[60] "strainDCLPA:conditionCAN:slevelMEDIUM"    
[61] "strainDCLPB:conditionCAN:slevelMEDIUM"    
[62] "strainWT:conditionCONTROL:slevelHIGH"     
[63] "strainDLON:conditionCONTROL:slevelHIGH"   
[64] "strainDKJ:conditionCONTROL:slevelHIGH"    
[65] "strainDCLPA:conditionCONTROL:slevelHIGH"  
[66] "strainDCLPB:conditionCONTROL:slevelHIGH"  
[67] "strainWT:conditionHS:slevelHIGH"          
[68] "strainDLON:conditionHS:slevelHIGH"        
[69] "strainDKJ:conditionHS:slevelHIGH"         
[70] "strainDCLPA:conditionHS:slevelHIGH"       
[71] "strainDCLPB:conditionHS:slevelHIGH"       
[72] "strainWT:conditionOS:slevelHIGH"          
[73] "strainDLON:conditionOS:slevelHIGH"        
[74] "strainDKJ:conditionOS:slevelHIGH"         
[75] "strainDCLPA:conditionOS:slevelHIGH"       
[76] "strainDCLPB:conditionOS:slevelHIGH"       
[77] "strainWT:conditionCAN:slevelHIGH"         
[78] "strainDLON:conditionCAN:slevelHIGH"       
[79] "strainDKJ:conditionCAN:slevelHIGH"        
[80] "strainDCLPA:conditionCAN:slevelHIGH"      
[81] "strainDCLPB:conditionCAN:slevelHIGH"

In [21]:
coef(glm.nb(tncnt ~ strain+condition+slevel,df))

(Intercept)   strainDLON    strainDKJ  strainDCLPA  strainDCLPB  conditionHS 
   6.1581095   -0.4827267   -6.1807390   -5.7541222   -2.5130191   -2.2260511 
 conditionOS conditionCAN    slevelLOW slevelMEDIUM   slevelHIGH 
  -0.5573401   -0.8043435    0.9119201    0.4046325           NA

In [30]:
coef(glm.nb(tncnt ~ strain:condition:slevel,df))

(Intercept) 
                                2.9123507 
     strainWT:conditionCONTROL:slevelnone 
                                4.0850168 
   strainDLON:conditionCONTROL:slevelnone 
                                2.4698482 
    strainDKJ:conditionCONTROL:slevelnone 
                               -4.2986450 
  strainDCLPA:conditionCONTROL:slevelnone 
                               -3.6054978 
  strainDCLPB:conditionCONTROL:slevelnone 
                                0.4287428 
          strainWT:conditionHS:slevelnone 
                                       NA 
        strainDLON:conditionHS:slevelnone 
                                       NA 
         strainDKJ:conditionHS:slevelnone 
                                       NA 
       strainDCLPA:conditionHS:slevelnone 
                                       NA 
       strainDCLPB:conditionHS:slevelnone 
                                       NA 
          strainWT:conditionOS:slevelnone 
                                       NA 
        strainDLON:conditionOS:slevelnone 
                                       NA 
         strainDKJ:conditionOS:slevelnone 
                                       NA 
       strainDCLPA:conditionOS:slevelnone 
                                       NA 
       strainDCLPB:conditionOS:slevelnone 
                                       NA 
         strainWT:conditionCAN:slevelnone 
                                       NA 
       strainDLON:conditionCAN:slevelnone 
                                       NA 
        strainDKJ:conditionCAN:slevelnone 
                                       NA 
      strainDCLPA:conditionCAN:slevelnone 
                                       NA 
      strainDCLPB:conditionCAN:slevelnone 
                                       NA 
      strainWT:conditionCONTROL:slevelLOW 
                                       NA 
    strainDLON:conditionCONTROL:slevelLOW 
                                       NA 
     strainDKJ:conditionCONTROL:slevelLOW 
                                       NA 
   strainDCLPA:conditionCONTROL:slevelLOW 
                                       NA 
   strainDCLPB:conditionCONTROL:slevelLOW 
                                       NA 
           strainWT:conditionHS:slevelLOW 
                                1.7743995 
         strainDLON:conditionHS:slevelLOW 
                                2.3708531 
          strainDKJ:conditionHS:slevelLOW 
                               -3.6054978 
        strainDCLPA:conditionHS:slevelLOW 
                               -3.6054978 
        strainDCLPB:conditionHS:slevelLOW 
                               -0.4484974 
           strainWT:conditionOS:slevelLOW 
                                3.5623113 
         strainDLON:conditionOS:slevelLOW 
                                2.3721213 
          strainDKJ:conditionOS:slevelLOW 
                               -3.6054978 
        strainDCLPA:conditionOS:slevelLOW 
                               -3.6054978 
        strainDCLPB:conditionOS:slevelLOW 
                                0.9162907 
          strainWT:conditionCAN:slevelLOW 
                                2.9882314 
        strainDLON:conditionCAN:slevelLOW 
                                2.4311790 
         strainDKJ:conditionCAN:slevelLOW 
                              -22.2149358 
       strainDCLPA:conditionCAN:slevelLOW 
                               -3.2000327 
       strainDCLPB:conditionCAN:slevelLOW 
                                1.6202488 
   strainWT:conditionCONTROL:slevelMEDIUM 
                                       NA 
 strainDLON:conditionCONTROL:slevelMEDIUM 
                                       NA 
  strainDKJ:conditionCONTROL:slevelMEDIUM 
                                       NA 
strainDCLPA:conditionCONTROL:slevelMEDIUM 
                                       NA 
strainDCLPB:conditionCONTROL:slevelMEDIUM 
                                       NA 
        strainWT:conditionHS:slevelMEDIUM 
                               

In [58]:

nbridge_signif = function(df) {
  # Extract the genename from the model
locus_tag=as.character(unique(df$locus_tag))
model = tryCatch(
    {
      x<- model.matrix(tncnt ~ strain/condition/slevel,df)
      y <- df$tncnt
      lambda_seq <- 10^seq(2, -2, by = -.1)
      t<- glm.nb(tncnt ~ strain/condition/slevel,df)$theta
      lambdas = NULL
      for (i in 1:5)
      {
        fit <- cv.glmnet(x, y, alpha = 0, lambda = lambda_seq,family = negative.binomial(theta = t),parallel=TRUE,intercept = FALSE)
        errors = data.frame(fit$lambda,fit$cvm)
        lambdas <- rbind(lambdas,errors)
      }
      # take mean cvm for each lambda
      lambdas <- aggregate(lambdas[, 2], list(lambdas$fit.lambda), mean)

      # select the best one
      bestindex = which(lambdas[2]==min(lambdas[2]))
      best_lambda = lambdas[bestindex,1]
      best_ridge <- glmnet(x, y, alpha = 0, lambda = best_lambda,family = negative.binomial(theta = t),intercept = FALSE)
      best_ridge
    },
    error=function(err) {
      status <<- err$message
      return(NULL)
    })
if (is.null(coef(model))) {
     namatrix=t(as.matrix(rep("NA",81)))
    row.names(namatrix)= locus_tag
    return (namatrix)} else {
    vals<- t(as.matrix(coef(model)))
    coeffs=t(as.matrix(vals[,-(which(colSums(vals) == 0))]))
    row.names(coeffs)= locus_tag
    return (coeffs)
  }
}

In [22]:
objects(model)

[1] "aic"               "boundary"          "call"             
 [4] "coefficients"      "contrasts"         "control"          
 [7] "converged"         "deviance"          "df.null"          
[10] "df.residual"       "effects"           "family"           
[13] "fitted.values"     "iter"              "linear.predictors"
[16] "method"            "model"             "null.deviance"    
[19] "prior.weights"     "qr"                "R"                
[22] "rank"              "residuals"         "SE.theta"         
[25] "terms"             "theta"             "twologlik"        
[28] "weights"           "xlevels"           "y"

In [25]:
#### Gene wise Fits

genewisefits <- function(tn,i,locuslist) {
  suppressWarnings(suppressMessages(library(tidyverse)))
  suppressWarnings(suppressMessages(require(pscl)))
  suppressWarnings(suppressMessages(require(MASS)))
  suppressWarnings(suppressMessages(require(reshape)))
  suppressWarnings(suppressMessages(require(glmnet)))
  g=locuslist[i]
  # Ordering factors so that intercept is always WT& control
  ord_condition<- c("CONTROL","HS","OS","CAN")
  ord_strain <- c("WT","DLON","DKJ","DCLPA","DCLPB")
  ord_slevel <- c("none","LOW","MEDIUM","HIGH")
  df<- tn%>%filter(locus_tag==g)%>%mutate(strain=fct_relevel(strain,ord_strain))%>%mutate(condition=fct_relevel(condition,ord_condition))%>%mutate(slevel=fct_relevel(slevel,ord_slevel))
  return(suppressWarnings(suppressMessages(nbridge_signif(df))))
  gc()
}


## parallel runs

no_cores <- 10
cl <- makeCluster(no_cores, type="PSOCK")
registerDoParallel(cl)

locusresults<-  list()
locuslist <- unique(tn$locus_tag)
locusresults <- foreach(i=2:length(locuslist)) %dopar% genewisefits(tn,i,locuslist)

In [26]:
### Model Unique Counts for each locus_tag and Store Log2 transformed coefficients

In [27]:
mod_data <- melt(locusresults)%>%dplyr::select(-L1)
# Select only gene,effect and coefficient
colnames(mod_data )<- c("locus_tag","effect","coeff")
mod_data  <- mod_data%>%mutate(coeff_log2value=as.numeric(coeff)/log(2))%>%filter(!effect %in% (1:80))

Warning message in mask$eval_all_mutate(quo):
"NAs introduced by coercion"


In [28]:
head(mod_data)

,locus_tag,effect,coeff,coeff_log2value
,<fct>,<chr>,<chr>,<dbl>
1,CCNA_00001,(Intercept),-13.2939514439527,-19.1791178
2,CCNA_00001,strainDLON,0.281919639733396,0.4067241
3,CCNA_00001,strainDKJ,-1.49438798291524,-2.1559461
4,CCNA_00001,strainDCLPA,-1.49841182493627,-2.1617513
5,CCNA_00001,strainDCLPB,-0.0983706575762541,-0.1419189
6,CCNA_00001,batch2,1.01784180089683,1.4684353


In [29]:
missedresults<- list()
suppressWarnings(suppressMessages(for (g in locuslist) {
    
    if(nrow(mod_data%>%filter(locus_tag==g))==0){
        
        missed <- melt(unique(mod_data$effect))
        missedresults[[g]] <- missed%>%mutate(locus_tag=g,effect=value,coeff="NA",coeff_log2value="NA")%>%dplyr::select(-value)   
    }   
}))
#Here L1 is dummy header given by melt function
suppressWarnings(suppressMessages(missed_data <- melt(missedresults)%>%dplyr::select(-L1)))

In [30]:
head(mod_data)

,locus_tag,effect,coeff,coeff_log2value
,<fct>,<chr>,<chr>,<dbl>
1,CCNA_00001,(Intercept),-13.2939514439527,-19.1791178
2,CCNA_00001,strainDLON,0.281919639733396,0.4067241
3,CCNA_00001,strainDKJ,-1.49438798291524,-2.1559461
4,CCNA_00001,strainDCLPA,-1.49841182493627,-2.1617513
5,CCNA_00001,strainDCLPB,-0.0983706575762541,-0.1419189
6,CCNA_00001,batch2,1.01784180089683,1.4684353


In [31]:
# Concatenating Model results and "NA's" of Missed Locus to a dataframe

In [32]:
unique_model_data<- rbind(mod_data,missed_data)

In [33]:
length(unique(unique_model_data$locus_tag))

[1] 4084

In [34]:
# Writing results to UC_zinb.tsv

In [35]:
write_tsv(unique_model_data, '../../models/TC_mid_production1_2_3_4_5_regularization.tsv')